# GDELT Raw Data File Collection

### This file will follow the steps below to collect the gdelt data
#### 1. This file will download the master file from official gdelt website. 
#### 2. Based on the user preference it will download the data from gdelt in compressed format.
#### 3. Un-compressing the downloaded files
#### 4. Combining the un-compressed files into one single csv file
#### 5. Filter the data based on Actor1Code = 'HLH' and Actor2Code ='HLH' for healthcare events and eventRootCode IN ('10','11','12','13','14')
#### 5. Push the combined csv file to NoSQL MongoDB

### Download the Master file

#### Check if the old master file already present in the directory. If it's present then delete the old file and download the latest one from GDELT

In [1]:
import time
#import tracemalloc
#tracemalloc.start()
start_time = time.process_time()

import os
file_path = r".\master\masterfilelist.txt"
if os.path.isfile(file_path):
  os.remove(file_path)
  print("Old Master file existed and has been deleted")
else:
  print("Old Master file not found!")

Old Master file existed and has been deleted


#### Download the master file which maps the raw data files

In [2]:
import wget
path_master = r".\master"
wget.download("http://data.gdeltproject.org/gdeltv2/masterfilelist.txt", out = path_master)

  1% [.                                                                       ]  1523712 / 82746692

  4% [..                                                                      ]  3440640 / 82746692

  5% [...                                                                     ]  4153344 / 82746692

  8% [.....                                                                   ]  6799360 / 82746692

 11% [........                                                                ]  9887744 / 82746692

...

 94% [...................................................................     ] 78102528 / 82746692

 97% [.....................................................................   ] 80322560 / 82746692

 99% [....................................................................... ] 82542592 / 82746692

100% [........................................................................] 82746692 / 82746692

'.\\master/masterfilelist.txt'

### Read the master file into a dataframe

In [3]:
import pandas as pd
header_list = ['A','B','C']
df = pd.read_csv(r".\master\masterfilelist.txt", sep= ' ', names = header_list)

#### Display the type of data that master file has and the amount of data it has

In [4]:
df.head()

,A,B,C
0,150383,297a16b493de7cf6ca809a7cc31d0b93,http://data.gdeltproject.org/gdeltv2/201502182...
1,318084,bb27f78ba45f69a17ea6ed7755e9f8ff,http://data.gdeltproject.org/gdeltv2/201502182...
2,10768507,ea8dde0beb0ba98810a92db068c0ce99,http://data.gdeltproject.org/gdeltv2/201502182...
3,149211,2a91041d7e72b0fc6a629e2ff867b240,http://data.gdeltproject.org/gdeltv2/201502182...
4,339037,dec3f427076b716a8112b9086c342523,http://data.gdeltproject.org/gdeltv2/201502182...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 774279 entries, 0 to 774278
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   A       774279 non-null  object
 1   B       774224 non-null  object
 2   C       774224 non-null  object
dtypes: object(3)
memory usage: 17.7+ MB


### Fetch all the export table files of event data
#### This code will download only the event data from GDELT

In [6]:
list_of_export_table = []

In [7]:
for i in df['C']:
    if 'export' in str(i):
        list_of_export_table.append(i)

In [8]:
len(list_of_export_table)

258073

In [9]:
#shows only first 5 records fro reference
list_of_export_table[:5]

['http://data.gdeltproject.org/gdeltv2/20150218230000.export.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218231500.export.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218233000.export.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150218234500.export.CSV.zip',
 'http://data.gdeltproject.org/gdeltv2/20150219000000.export.CSV.zip']

### Download the x number of files from today
#### This will download all the files going back from the day (Exact time) the code is run

In [10]:
print("ENTER THE NUMEBR OF FILES YOU WANT TO DOWNLOAD")
num = input()

ENTER THE NUMEBR OF FILES YOU WANT TO DOWNLOAD
100


In [11]:
path_zip = r'.\zip_data'

In [12]:
import shutil
if len(os.listdir(path_zip)) == 0:
    print("Directory is empty")
else:
    shutil.rmtree(path_zip)

In [13]:
os.mkdir(path_zip)

In [14]:
for i in list_of_export_table[-int(num):]:
    wget.download(i, out = path_zip)

100% [..............................................................................] 92549 / 92549

In [15]:
len(os.listdir(path_zip))

100

### Unzip the downloaded files to a seperate CSV folder

In [16]:
import shutil

In [17]:
path_csv = r'.\csv_data'

In [18]:
import shutil
if len(os.listdir(path_csv)) == 0:
    print("Directory is empty")
else:
    shutil.rmtree(path_csv)

In [19]:
os.mkdir(path_csv)

In [20]:
file_names = os.listdir(path_zip)

In [21]:
for i in file_names:
    shutil.unpack_archive(path_zip + '\\' + i, path_csv)

### Combine all the CSV into one

In [22]:
header = ['GLOBALEVENTID', 
            'SQLDATE', 
            'MonthYear',
            'Year', 
            'FractionDate',
            'Actor1Code', 
            'Actor1Name', 
            'Actor1CountryCode',
            'Actor1KnownGroupCode',
            'Actor1EthnicCode',
            'Actor1Religion1Code',
            'Actor1Religion2Code',
            'Actor1Type1Code',
            'Actor1Type2Code',
            'Actor1Type3Code',
            'Actor2Code',
            'Actor2Name',
            'Actor2CountryCode',
            'Actor2KnownGroupCode',
            'Actor2EthnicCode',
            'Actor2Religion1Code',
            'Actor2Religion2Code',
            'Actor2Type1Code',
            'Actor2Type2Code',
            'Actor2Type3Code',
            'IsRootEvent',
            'EventCode',
            'EventBaseCode',
            'EventRootCode',
            'QuadClass',
            'GoldsteinScale',
            'NumMentions',
            'NumSources',
            'NumArticles',
            'AvgTone',
            'Actor1Geo_Type',
            'Actor1Geo_FullName',
            'Actor1Geo_CountryCode',
            'Actor1Geo_ADM1Code',
            'Actor1Geo_ADM2Code',
            'Actor1Geo_Lat',
            'Actor1Geo_Long',
            'Actor1Geo_FeatureID',
            'Actor2Geo_Type',
            'Actor2Geo_FullName',
            'Actor2Geo_CountryCode',
            'Actor2Geo_ADM1Code',
            'Actor2Geo_ADM2Code',
            'Actor2Geo_Lat',
            'Actor2Geo_Long',
            'Actor2Geo_FeatureID',
            'ActionGeo_Type',
            'ActionGeo_FullName',
            'ActionGeo_CountryCode',
            'ActionGeo_ADM1Code',
            'ActionGeo_ADM2Code',
            'ActionGeo_Lat',
            'ActionGeo_Long',
            'ActionGeo_FeatureID',
            'DATEADDED',
            'SOURCEURL']


In [23]:
len(header)

61

In [24]:
import glob
files = os.path.join(path_csv, "*.csv")

In [25]:
files

'.\\csv_data\\*.csv'

In [26]:
files = glob.glob(files)

In [27]:
final_csv = pd.concat([pd.read_csv(f, sep = '\t', names = header) for f in files])

In [28]:
final_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114833 entries, 0 to 1316
Data columns (total 61 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   GLOBALEVENTID          114833 non-null  int64  
 1   SQLDATE                114833 non-null  int64  
 2   MonthYear              114833 non-null  int64  
 3   Year                   114833 non-null  int64  
 4   FractionDate           114833 non-null  float64
 5   Actor1Code             103614 non-null  object 
 6   Actor1Name             103614 non-null  object 
 7   Actor1CountryCode      64340 non-null   object 
 8   Actor1KnownGroupCode   1404 non-null    object 
 9   Actor1EthnicCode       663 non-null     object 
 10  Actor1Religion1Code    1422 non-null    object 
 11  Actor1Religion2Code    355 non-null     object 
 12  Actor1Type1Code        49726 non-null   object 
 13  Actor1Type2Code        3613 non-null    object 
 14  Actor1Type3Code        78 non-null    

In [29]:
len(final_csv)

114833

#### Run the query on the combined data to filter the data based on healthcare events and event root code

In [30]:
import pandasql as pds

In [31]:
query = """SELECT * FROM final_csv WHERE EventRootCode IN ('10','11','12','13','14') AND (ACTOR1CODE = 'HLH' OR 'ACTOR2CODE' = 'HLH') """

In [32]:
new_df = pds.sqldf(query,globals())

In [33]:
new_df.head()

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1060046088,20220824,202208,2022,2022.6411,HLH,HOSPITAL,None,None,None,...,4,"Wellington, New Zealand (general), New Zealand",NZ,NZ00,22391,-41.3000,174.783,-1521348,20220824221500,https://kitchener.ctvnews.ca/grand-river-hospi...
1,1060047085,20220824,202208,2022,2022.6411,HLH,DOCTOR,None,None,None,...,1,United Kingdom,UK,UK,None,54.0000,-4.000,UK,20220824223000,https://vancouverisland.ctvnews.ca/british-col...
2,1060050113,20220824,202208,2022,2022.6411,HLH,NURSE,None,None,None,...,3,"Asheville, North Carolina, United States",US,USNC,NC021,35.6009,-82.554,1018864,20220824230000,https://www.healthleadersmedia.com/nursing/mis...
3,1060050114,20220824,202208,2022,2022.6411,HLH,NURSE,None,None,None,...,3,"Asheville, North Carolina, United States",US,USNC,NC021,35.6009,-82.554,1018864,20220824230000,https://www.healthleadersmedia.com/nursing/mis...
4,1060050116,20220824,202208,2022,2022.6411,HLH,NURSE,None,None,None,...,3,"Asheville, North Carolina, United States",US,USNC,NC021,35.6009,-82.554,1018864,20220824230000,https://www.healthleadersmedia.com/nursing/mis...


In [34]:
len(new_df)

69

In [35]:
new_df.EventRootCode.unique()

array([11, 12, 10, 14, 13], dtype=int64)

In [36]:
new_df.head()

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_ADM2Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,1060046088,20220824,202208,2022,2022.6411,HLH,HOSPITAL,None,None,None,...,4,"Wellington, New Zealand (general), New Zealand",NZ,NZ00,22391,-41.3000,174.783,-1521348,20220824221500,https://kitchener.ctvnews.ca/grand-river-hospi...
1,1060047085,20220824,202208,2022,2022.6411,HLH,DOCTOR,None,None,None,...,1,United Kingdom,UK,UK,None,54.0000,-4.000,UK,20220824223000,https://vancouverisland.ctvnews.ca/british-col...
2,1060050113,20220824,202208,2022,2022.6411,HLH,NURSE,None,None,None,...,3,"Asheville, North Carolina, United States",US,USNC,NC021,35.6009,-82.554,1018864,20220824230000,https://www.healthleadersmedia.com/nursing/mis...
3,1060050114,20220824,202208,2022,2022.6411,HLH,NURSE,None,None,None,...,3,"Asheville, North Carolina, United States",US,USNC,NC021,35.6009,-82.554,1018864,20220824230000,https://www.healthleadersmedia.com/nursing/mis...
4,1060050116,20220824,202208,2022,2022.6411,HLH,NURSE,None,None,None,...,3,"Asheville, North Carolina, United States",US,USNC,NC021,35.6009,-82.554,1018864,20220824230000,https://www.healthleadersmedia.com/nursing/mis...


#### Push the data into a new CSV file

In [37]:
%%time
new_df.to_csv(r".\GDELT_data.csv", index = False)

CPU times: total: 15.6 ms
Wall time: 188 ms


### Export data to MongoDB collection

In [38]:
import pymongo

In [39]:
#Create connection to the mongodb client
client = pymongo.MongoClient("mongodb://localhost:27017")

In [40]:
client.list_database_names()

['GDELT', 'admin', 'config', 'local']

In [41]:
db = client['GDELT']
db.list_collection_names()

['Balanced_Data_All', 'Balanced_Data_No_Protest_Code_Significant']

In [42]:
if('raw_data_files' in db.list_collection_names()):
    db.raw_data_event.drop()
else:
    print("Collection does not exists!")

Collection does not exists!


In [43]:
raw_data_files = db['raw_data_files']

In [44]:
#read the generated CSV
data = pd.read_csv(r".\GDELT_data.csv")

In [45]:
import json
final_data = json.loads(data.to_json(orient='records'))

In [46]:
%%time
raw_data_files.insert_many(final_data)
end_time = time.process_time()

CPU times: total: 0 ns
Wall time: 608 ms


In [47]:
total_time = end_time - start_time

In [48]:
total_time

32.53125